# get data body in spark

In [1]:
import re
data = sc.wholeTextFiles('data/reut2-02*')
newsArticles = data.map(lambda x:x[1]).flatMap(lambda x:x.split('<BODY>')[1:]).map(lambda x:x.split('</BODY>')[0])\
                   .map(lambda x:re.sub(' +', ' ', x.replace('\n', ' ')))

newsArticles.take(3)

['If the dollar goes the way of Wall Street, Japanese will finally move out of dollar investments in a serious way, Japan investment managers say. The Japanese, the dominant foreign investors in U.S. Dollar securities, have already sold U.S. Equities. But "if the dollar falls steeply, which did not happen yesterday, Japanese investors will definitely try to withdraw significant funds from U.S. Shares," said Akira Kawakami, deputy manager of Nomura Investment Trust and Management Co Ltd\'s international investment department. An unstable, lower dollar would also affect Japanese investment in U.S. Bonds. "Japan-U.S. Interest rate differentials, which currently look wide enough, mean nothing in the absence of dollar stability," said Kawakami. U.S. Bonds could benefit due to a gloomy economic picture following the estimated huge losses in stocks by major U.S. Institutional and individual investors, he said. The effect should be to rule out any U.S. Interest rate rise. But most Japanese inv

# (1) Given the Reuters-21578 dataset, please calculate all k-shingles and output the set representation of the text dataset as a matrix.

In [2]:
k = 3
shingles = newsArticles.flatMap(lambda x:[x[i:i+k] for i in range(len(x)-k+1)]).distinct()

shingles.take(5)

['If ', 'f t', ' th', 'oll', 'lla']

In [3]:
shingles_count = shingles.count()
articles_count = newsArticles.count()
print(shingles_count, 'different shingles.')
print(articles_count, 'different articles.')

13759 different shingles.
1237 different articles.


In [4]:
newsArticles = newsArticles.collect()
kShinglesMatrix = shingles.map(lambda s:[1 if s in a else 0 for a in newsArticles]).collect()

In [5]:
with open('result/kShinglesMatrix.txt', 'w') as result:
    for row in kShinglesMatrix:
        result.write(str(row) + '\n')

# (2) Given the set representation, compute the minhash signatures of all documents using MapReduce.

In [24]:
# local version
h = 100

import random

minHashSignatures = list()
for i in range(h):
    rndIdx = random.sample(range(shingles_count), shingles_count)
    minHash = [-1]*articles_count
    while -1 in minHash:
        row = rndIdx.pop()
        for j in range(len(minHash)):
            if (minHash[j] == -1 or minHash[j] > row) and kShinglesMatrix[row][j] == 1:
                minHash[j] = row
    minHashSignatures.append(minHash)

In [21]:
import sys
import random

def biggerThanNFirstPrime(N):
    p = 2
    while True:
        isPrime = True
        for i in range(2,p//2+1):
            if(p%i==0):
                isPrime = False
                break
        if isPrime and p > N:
            return p
        else:
            p+=1

h = 100
a = [random.randint(1, sys.maxsize) for i in range(h)]
b = [random.randint(1, sys.maxsize) for i in range(h)]
p = biggerThanNFirstPrime(articles_count)
N = articles_count

def rowHash(row, a, b, p, N):
    return ((a*row+b)%p)%N

In [22]:
minHashSignatures = list()
for i in range(h):
    minHashSignatures.append(sc.parallelize(kShinglesMatrix).zipWithIndex()\
                               .map(lambda x:[rowHash(x[1], a[i], b[i], p ,N) if c == 1 else sys.maxsize for c in x[0]])\
                               .reduce(lambda x, y:[Mx if Mx < My else My for Mx, My in zip(x, y)]))

In [9]:
with open('result/minHashSignatures.txt', 'w') as result:
    for row in minHashSignatures:
        result.write(str(row) + '\n')

# Jaccard distance 1 to anothers

In [25]:
jaccardDistance = list()
first = list()
for i in range(len(minHashSignatures)):
    first.append(minHashSignatures[i][0])
first = set(first)
    
for i in range(len(minHashSignatures[0])):
    other = list()
    for j in range(len(minHashSignatures)):
        other.append(minHashSignatures[j][i])
    print(len(first & set(other))/len(first | set(other)))

1.0
0.16666666666666666
0.1875
0.3835616438356164
0.3611111111111111
0.1951219512195122
0.21518987341772153
0.2625
0.30985915492957744
0.17857142857142858
0.18181818181818182
0.46875
0.32432432432432434
0.2125
0.4264705882352941
0.24050632911392406
0.1744186046511628
0.30666666666666664
0.6779661016949152
0.7142857142857143
0.39705882352941174
0.07526881720430108
0.21518987341772153
0.17073170731707318
0.1744186046511628
0.5230769230769231
0.32051282051282054
0.3424657534246575
0.3150684931506849
0.4393939393939394
0.4492753623188406
0.47692307692307695
0.43478260869565216
0.34285714285714286
0.18072289156626506
0.1625
0.36764705882352944
0.1590909090909091
0.12790697674418605
0.4461538461538462
0.47692307692307695
0.4225352112676056
0.07954545454545454
0.33783783783783783
0.175
0.3382352941176471
0.2702702702702703
0.24675324675324675
0.21518987341772153
0.15476190476190477
0.16470588235294117
0.3561643835616438
0.1839080459770115
0.225
0.2876712328767123
0.18072289156626506
0.3066666